# INFO 2950 Final Project

In [2]:
import pandas as pd
import seaborn

In [3]:
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

reserach question:

In [21]:
table1 = pd.read_csv('./Data_2019/Incidents_Offenses_Victims_and_Known_Offenders_by_Bias_Motivation_2019.csv')

table4 = pd.read_csv('./Data_2019/Offenses_Offense_Type_by_Bias_Motivation_2019 v2.csv')

table4

,Bias motivation,Total_offenses,Murder_and_nonnegligent_manslaughter,Rape,Aggravated_assault,Simple_assault,Intimidation,"Human_Trafficking,_Commercial_Sex_Acts",Other,Robbery,Burglary,Larceny_theft,Motor_vehicle_theft,Arson,Destruction_damage_vandalism,Other.1,Crimes_against_society
0,Total,"8,559",51,30,"1,158","2,023","2,206",3,41,125,114,284,19,60,"2,152",57,236
1,Single-Bias Incidents,"8,302",51,30,"1,139","1,976","2,134",3,41,125,109,282,19,55,"2,050",57,231
2,Race/Ethnicity/Ancestry:,"4,784",31,12,726,"1,225","1,385",0,13,62,46,149,13,16,901,35,170
3,Anti-White,755,0,6,108,240,143,0,3,16,10,56,4,1,72,15,81
4,Anti-Black or African American,"2,314",1,2,344,546,797,0,8,13,17,19,3,9,516,5,34
5,Anti-American Indian or Alaska Native,126,0,1,5,26,11,0,1,4,5,32,4,1,15,3,18
6,Anti-Asian,205,4,1,30,78,50,0,0,4,3,3,1,2,25,0,4
7,Anti-Native Hawaiian or Other Pacific Islander,25,0,0,4,4,1,0,0,0,0,4,0,0,1,2,9
8,"Anti-Multiple Races, Group",171,3,0,17,27,57,0,0,1,0,6,0,0,53,4,3
9,Anti-Arab,122,0,0,20,36,37,0,0,1,1,3,0,0,21,0,3
